<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 6
<br>
<b>Deadline:</b> March 24, 2021 (Wednesday) 23:00
</div>

# Exercise 4.2 Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
skip_training = False  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is ../data


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

0it [00:00, ?it/s]

Extracting ../data/ratings.zip to ../data


In [8]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [106]:
def tabular_preview(ratings, n=15):
    """Creates a cross-tabular view of users vs movies."""
    
    user_groups = ratings.groupby('user_id')['rating'].count()
    top_users = user_groups.sort_values(ascending=False)[:n]

    movie_groups = ratings.groupby('movie_id')['rating'].count()
    top_movies = movie_groups.sort_values(ascending=False)[:n]

    top = (
        ratings.
        join(top_users, rsuffix='_r', how='inner', on='user_id').
        join(top_movies, rsuffix='_r', how='inner', on='movie_id'))

    return pd.crosstab(top.user_id, top.movie_id, top.rating, aggfunc=np.sum)

ratings = pd.DataFrame({'user_id': trainset[:][0], 'movie_id': trainset[:][1], 'rating': trainset[:][2]})
tabular_preview(ratings)

movie_id,1,7,50,56,98,100,121,127,174,181,258,286,288,294,300
user_id,,,,,,,,,,,,,,,
7,NaN,5.0,NaN,5.0,4.0,5.0,5.0,5.0,5.0,3.0,4.0,4.0,4.0,1.0,4.0
13,3.0,2.0,5.0,5.0,4.0,5.0,5.0,5.0,4.0,5.0,4.0,3.0,1.0,2.0,1.0
94,4.0,4.0,5.0,5.0,4.0,5.0,2.0,NaN,4.0,4.0,5.0,4.0,3.0,NaN,NaN
181,3.0,4.0,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,1.0,4.0,2.0,3.0
234,3.0,2.0,4.0,3.0,4.0,4.0,NaN,NaN,3.0,3.0,NaN,3.0,3.0,3.0,3.0
276,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,NaN,4.0,4.0,NaN
279,3.0,NaN,3.0,4.0,NaN,4.0,4.0,NaN,4.0,3.0,NaN,NaN,3.0,2.0,NaN
303,5.0,NaN,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,4.0,5.0,4.0,4.0,1.0
308,4.0,4.0,5.0,5.0,3.0,5.0,NaN,4.0,4.0,4.0,NaN,NaN,4.0,3.0,NaN


In [107]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        super(RecommenderSystem, self).__init__()
        
        n_factors = 50
        self.users = nn.Embedding(n_users + 1, n_factors, sparse=False)
        self.movies = nn.Embedding(n_items + 1, n_factors, sparse=False)
        self.embedding_drop = nn.Dropout(0.02)
        
        self.features = nn.Sequential(
            nn.Linear(100, 200, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.02),
        )
        self.fc = nn.Linear(200, 1)
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        features = torch.cat([self.users(user_ids), self.movies(item_ids)], dim=1)
        x = self.embedding_drop(features)
        x = self.features(x)
        out = torch.sigmoid(self.fc(x))
        
        return torch.squeeze(out)


You can test the shapes of the model outputs using the function below.

In [108]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [109]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

In [72]:
# Create the model
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [140]:
#print(model)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
# training loop parameters
lr = 1e-3
wd = 1e-5
epochs = 10


set_length = len(trainset)

if not skip_training:
    criterion = nn.MSELoss()
    model.zero_grad()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    
    for epoch in range(epochs):
        stats = {'epoch': epoch + 1, 'total': epochs}
        running_loss = 0
        model.train()
        for batch_num, training_batch in enumerate(trainloader):
            users, movies, ratings = training_batch
            ratings = ratings.to(torch.float32)
            optimizer.zero_grad()
            output = model(users, movies)
            loss = criterion(output, ratings)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_loss = running_loss / set_length
            stats["loss"] = epoch_loss
            
        print("{epoch}/{total} loss: {loss}".format(**stats))
    model.eval()

1/10 loss: 0.2399921123445034
2/10 loss: 0.23999211245775223
3/10 loss: 0.2399921120584011
4/10 loss: 0.23999211239218712
5/10 loss: 0.23999211245179175
6/10 loss: 0.23999211184382438
7/10 loss: 0.23999211292266845
8/10 loss: 0.23999211228489875
9/10 loss: 0.23999211260676384
10/10 loss: 0.2399921121299267


In [76]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

Do you want to save the model (type yes to confirm)? yes
Model saved to recsys.pth.


In [77]:
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

In [78]:
# This cell is reserved for grading

In [79]:
# This cell is reserved for grading

In [80]:
# This cell is reserved for grading

In [81]:
# This cell is reserved for grading

In [82]:
# This cell is reserved for grading

In [83]:
# This cell is reserved for grading

In [84]:
# This cell is reserved for grading